In [1]:
!pip install transformers PyPDF2 bertopic


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 103.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 135.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import bertopic
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import warnings
from transformers import pipeline
import re
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 500)


In [3]:
# import OS module
import os
from pathlib import Path

def extract_text_from_directory(directory_loc):
    # Create an empty data frame
    df = pd.DataFrame(columns=['file', 'text'])

    files = Path(directory_loc).iterdir()

    print("Files and directories in '", directory_loc, "' :")
    # prints all files
    print(files)

    # Iterate over the PDF files
    for text_file in files:
        # Open the text file
        f = open(text_file, 'r')
        text = f.read()
        # Add the file name and the text to the data frame
        df = df.append({'file': text_file.name, 'text': text}, ignore_index=True)

    # Return the data frame
    return df


In [5]:
path='qa-input-data'
#files = Path(path).glob("WHR+22.pdf")
df = extract_text_from_directory(path)
df

Files and directories in ' qa-input-data ' :
<generator object Path.iterdir at 0x7f81d7e76f10>


,file,text
0,Data-Visualization.txt,"Data visualization\nLearn how to visualize data in Improvado reporting, configure and create reports setup connection with third-party visualization platforms\nImprovado reporting\nLearn how to visualize your data in Improvado reporting platform, configure and create reports\n\n\nSharing a report\nLearn how to share your report in Improvado reportin\n\nVisualization platforms\nLearn how to connect your data to visualization platforms\n\n\nWidget types\nLearn how to use different types of wid..."
1,pdf-export.txt,"You can export your report as a PDF file in two ways:\n\t•\tby clicking the “Export to PDF” at the top right corner\n\t•\tby selecting the “Schedule export to PDF” item in the “Report settings” menu.\nHow to export your report as a PDF file by clicking the“Export to PDF”\nYou can choose, would you prefer to turn into PDF the whole report with all tabs created or select a tab needed.\n\nPress the variant you choose, and the PDF generation will begin.\nBy default, this PDF includes only report..."
2,Welcome.txt,"Set up data extraction\nStep 1 Select accounts\nStep 2 Choose extraction templates\n\t•\tExtraction template is a feature that allows you to simplify the data extraction process by using the predefined set of metrics/dimensions and the most common settings for them.\nStep 3 Configure extraction template\n\t•\tScheduling settings, such as extraction time, time zone, lookback window\n\t•\tSet of dimensions and metrics\nStep 4 Check the result.\nLoad your data\nConnect the necessary destination..."
3,Extract.txt,"Extracting data\n\nLearn more about setting up connections in Improvado and how data is extracted\nImprovado Data Dictionary\nHere is an extensive overview of the full list of all common report types by data sources. This list includes such fields as metrics, properties, dimensions, etc. There are also many custom reports we don’t share here. Talk to our experts to get more details on it.\n\nConnect your data\nLearn how to connect your data to Improvado\n\nData sources\nLearn which data sour..."
4,share-report-with-private-link.txt,"You can share your report by generating a private link and sending it to your addressee. Select the Sharing Settings option in the Report Settings menu.\n\nThe Sharing Settings menu will appear. Click the Share by link button on the right side of the pop-up window.\n\nYou will see the link, which you can copy and send to your clients.\n\nThe Allow to reveal provider checkbox defines how detailed will be your report for a quest. Tick it to give more information about the data structure or set..."
5,Where-to-find-connected-source-config.txt,"Where to Find Your Connected Source Configuration\nChoose the required connection from the connected sources list to dive into its settings and see the related extraction orders list.\n \nHere you see a Page header and two tabs:\n\t•\tOverview (opens by default)\n\t•\tSettings\nPage header\nThe Page header shows brief details about the connected data source, business account, and your credentials validity status.\n \nYou can set up data extraction only for an active connection with at least ..."
6,Connect.txt,"Connect your data\nLearn how to connect your data to Improvado\nThe Data sources tab is one of the most important parts of the Improvado reporting platform.\nData source (platform, statistics provider, integration) is a system that provides marketing data.\n‍Here you can find information about all the available integrations.\n \nTo open this page click on the Data sources item in the left menu, or use a direct link. Each data source is represented by a tile with a platform logo and name.\nCo..."
7,Data-Sources.txt,Data sources\nLearn which data sources you can set up in Improvado\nGoogle Ads\nLearn how to set up Google Ads connection\n\n \n \nGoogle Display & Video 360\nLearn how to set up Google Display & Video 360 connection\n\n \n \nGoogle Search Console\nLearn how to set up Google Search Console 

In [6]:
# Import the re module for regular expressions
import re

def preprocess_text(text_list):
    # Initialize a list to store the pre-processed text
    processed_text = []

    # Iterate over the text in the list
    for text in text_list:
      num_words = len(text.split(" "))
      if num_words > 5:
        processed_text.append(text)

    # Return the pre-processed text
    return processed_text


def remove_short_sentences(df):
  df['sentences'] = df['sentences'].apply(preprocess_text)
  return df

In [7]:
df['sentences'] = df['text'].apply(lambda long_str: long_str.replace("\n", " ").split("."))
df = remove_short_sentences(df)
df

,file,text,sentences
0,Data-Visualization.txt,"Data visualization\nLearn how to visualize data in Improvado reporting, configure and create reports setup connection with third-party visualization platforms\nImprovado reporting\nLearn how to visualize your data in Improvado reporting platform, configure and create reports\n\n\nSharing a report\nLearn how to share your report in Improvado reportin\n\nVisualization platforms\nLearn how to connect your data to visualization platforms\n\n\nWidget types\nLearn how to use different types of wid...","[Data visualization Learn how to visualize data in Improvado reporting, configure and create reports setup connection with third-party visualization platforms Improvado reporting Learn how to visualize your data in Improvado reporting platform, configure and create reports Sharing a report Learn how to share your report in Improvado reportin Visualization platforms Learn how to connect your data to visualization platforms Widget types Learn how to use different types of widgets ]"
1,pdf-export.txt,"You can export your report as a PDF file in two ways:\n\t•\tby clicking the “Export to PDF” at the top right corner\n\t•\tby selecting the “Schedule export to PDF” item in the “Report settings” menu.\nHow to export your report as a PDF file by clicking the“Export to PDF”\nYou can choose, would you prefer to turn into PDF the whole report with all tabs created or select a tab needed.\n\nPress the variant you choose, and the PDF generation will begin.\nBy default, this PDF includes only report...","[You can export your report as a PDF file in two ways: \t•\tby clicking the “Export to PDF” at the top right corner \t•\tby selecting the “Schedule export to PDF” item in the “Report settings” menu, How to export your report as a PDF file by clicking the“Export to PDF” You can choose, would you prefer to turn into PDF the whole report with all tabs created or select a tab needed, Press the variant you choose, and the PDF generation will begin, By default, this PDF includes only report da..."
2,Welcome.txt,"Set up data extraction\nStep 1 Select accounts\nStep 2 Choose extraction templates\n\t•\tExtraction template is a feature that allows you to simplify the data extraction process by using the predefined set of metrics/dimensions and the most common settings for them.\nStep 3 Configure extraction template\n\t•\tScheduling settings, such as extraction time, time zone, lookback window\n\t•\tSet of dimensions and metrics\nStep 4 Check the result.\nLoad your data\nConnect the necessary destination...","[Set up data extraction Step 1 Select accounts Step 2 Choose extraction templates \t•\tExtraction template is a feature that allows you to simplify the data extraction process by using the predefined set of metrics/dimensions and the most common settings for them, Step 3 Configure extraction template \t•\tScheduling settings, such as extraction time, time zone, lookback window \t•\tSet of dimensions and metrics Step 4 Check the result, Load your data Connect the necessary destination We ha..."
3,Extract.txt,"Extracting data\n\nLearn more about setting up connections in Improvado and how data is extracted\nImprovado Data Dictionary\nHere is an extensive overview of the full list of all common report types by data sources. This list includes such fields as metrics, properties, dimensions, etc. There are also many custom reports we don’t share here. Talk to our experts to get more details on it.\n\nConnect your data\nLearn how to connect your data to Improvado\n\nData sources\nLearn which data sour...","[Extracting data Learn more about setting up connections in Improvado and how data is extracted Improvado Data Dictionary Here is an extensive overview of the full list of all common report types by data sources, This list includes such fields as metrics, properties, dimensions, etc, There are also many custom reports we don’t share here, Talk to our experts to get more details on it, Connect your data Learn how

In [8]:
"""
We have a corpus with various sentences. Then, for a given search query,
we want to find the most similar sentence in the document.
This script outputs the similarity score for all sentences in the document.
"""

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2') 
from sklearn.metrics.pairwise import cosine_similarity
cosine_threshold = 0.5 # set threshold for cosine similarity value

queries = ['types of widgets'] #search query

print("\nSemantic Search Results")

results = []
for i, document in enumerate(df['sentences']):
  sentence_embeddings = model.encode(document)
  query_embedding    = model.encode(queries)
  for j, sentence_embedding in enumerate(sentence_embeddings):
    distance = cosine_similarity(sentence_embedding.reshape((1,-1)), query_embedding.reshape((1,-1)))[0][0]
    
    sentence = df['sentences'].iloc[i][j]
    results += [(i, sentence, distance)]
results = sorted(results, key=lambda x: x[2], reverse=True)


Semantic Search Results


In [9]:
print(f"Query: {queries}")
print(f"Order by most relevant sentences in corpus:\n")

for idx, sentence, distance in results:
        if (distance > cosine_threshold):
                print(f"{sentence.strip()}, \n{df['file'].iloc[idx]}\nCosine Score: {distance:.4f})")
                print('-----------------------')
                df = df.append({'file': df['file'].iloc[idx], 'query': queries, 'sentence':sentence,
                 'cosine_score': (distance)}, ignore_index=True)

Query: ['types of widgets']
Order by most relevant sentences in corpus:

Learn how to use different types of widgets, 
widget-types.txt
Cosine Score: 0.8689)
-----------------------
Table widget is one of four widget types you can use to present data in widget 2, 
table.txt
Cosine Score: 0.6054)
-----------------------
0 Key Functions: Learn how to use Widget 2, 
widget-types.txt
Cosine Score: 0.5537)
-----------------------


In [10]:
df[~df['sentence'].isnull()][['file', 'query', 'sentence', 'cosine_score']].sort_values(by = 'cosine_score', ascending = False)

,file,query,sentence,cosine_score
15,widget-types.txt,[types of widgets],Learn how to use different types of widgets,0.868857
16,table.txt,[types of widgets],Table widget is one of four widget types you can use to present data in widget 2,0.605435
17,widget-types.txt,[types of widgets],0 Key Functions: Learn how to use Widget 2,0.553745


In [11]:
# extract all the sentences from results that have a cosine similarity score larger than the threshold 
# and put in a list
texts = []
for idx, sentence, distance in results:
    if distance > cosine_threshold:
        text = sentence
        texts.append(text)
#turn the list to string
final_text = "".join(texts)

In [12]:
from transformers import BertForQuestionAnswering, AutoTokenizer
modelname = 'deepset/bert-base-cased-squad2'
model_qa = BertForQuestionAnswering.from_pretrained(modelname)
#initalized the token, and convert the list of strings (tokens) into a list of integers (token IDs).  This is done using an internal dictionary that contains every token understood by Bert.
tokenizer = AutoTokenizer.from_pretrained(modelname)

In [13]:
nlp = pipeline('question-answering', model=model_qa, tokenizer=tokenizer)

context = final_text

nlp({
    'question': 'what are the different types of widgets ?',
    'context': context
})

{'score': 0.03442991524934769,
 'start': 66,
 'end': 83,
 'answer': 'four widget types'}